In [2]:
import pandas as pd 
import requests 
import json 
import matplotlib.pyplot as plt
import numpy as np 

# make a GET request to the endpoint and store the response in a variable called response
response = requests.get("https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=USD&e=BTCE&limit=2000&aggregate=1&timestamp='2017-04-01'")
bitcoin_dict = response.json()
columns = ["close","high","low","open","time","volumefrom","volumeto"]

In [3]:
# Generate a dataframe named bitcoin_data with ohlcv data in 1 hour 
bitcoin_data = pd.DataFrame(bitcoin_dict["Data"],columns=columns)


In [4]:
# Create the column Date that convert the time column in a timestamp object 
bitcoin_data["Date"] = pd.to_datetime(bitcoin_data["time"], unit="s")
bitcoin_data = bitcoin_data.drop("time",axis=1)



In [5]:
bitcoin_data = bitcoin_data[["Date","open","high","low","close","volumefrom","volumeto"]]

In [6]:
# Calculate the twenty day simple moving average and the fifty day simply moving average. These will be the indicators of the strategy
bitcoin_data["20d"] = np.round(bitcoin_data["close"].rolling(window = 20, center = False).mean(), 2)
bitcoin_data["50d"] = np.round(bitcoin_data["close"].rolling(window = 50, center = False).mean(), 2)


bitcoin_data.tail(10)


,Date,open,high,low,close,volumefrom,volumeto,20d,50d
1991,2017-07-28 00:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1992,2017-07-28 01:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1993,2017-07-28 02:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1994,2017-07-28 03:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1995,2017-07-28 04:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1996,2017-07-28 05:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1997,2017-07-28 06:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1998,2017-07-28 07:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
1999,2017-07-28 08:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0
2000,2017-07-28 09:00:00,2546.0,2546.0,2546.0,2546.0,0.0,0.0,2546.0,2546.0


In [7]:
bitcoin_data["H-L"] = bitcoin_data["high"] - bitcoin_data["low"]
bitcoin_data["H-YC"] = abs(bitcoin_data["high"] - bitcoin_data["close"].shift(1))
bitcoin_data["L-YC"] = abs(bitcoin_data["low"] - bitcoin_data["close"].shift(1))

In [8]:
def backTesting():
    x = 0
    count=0
    lastBoughtFor = 0
    stance = "none"
    totalProfit = 0
     #datetime.datetime.fromtimestamp(float(bitcoin_OHLCV.index)).strftime("%Y-%m-%d")))
    while x < len(bitcoin_data):
        if stance == "none":
            if (bitcoin_data["20d"][x] > bitcoin_data["50d"][x]): #& (bitcoin_data["ADX"][x] >20): #& (bitcoin_data["close"][x] >bitcoin_data["20d"][x]): # oversold, let´s buy
                print("starting time", str(bitcoin_data["Date"][x]))
                stance = "holding"
                print("Buying BTC @", bitcoin_data["close"][x])
                lastBoughtFor = bitcoin_data["close"][x]
                
        elif stance == "holding":
            if bitcoin_data["20d"][x] < bitcoin_data["50d"][x]:
                stance = "none"
                print("Selling BTC @", bitcoin_data["close"][x])
                fees = (.002)*(bitcoin_data["close"][x] + lastBoughtFor)
                print("Fee for this was",fees)
                print("finish time", str(bitcoin_data["Date"][x]))
                print("Profit on this trade:",bitcoin_data["close"][x] - lastBoughtFor - fees) # if we lost money this is negative 
                totalProfit += (bitcoin_data["close"][x] - lastBoughtFor - fees)
                count +=1
            
        
        x +=1
    print("total profit of", totalProfit)
    print(count)
backTesting()

starting time 2017-05-08 02:00:00
Buying BTC @ 1527.0
Selling BTC @ 1640.0
Fee for this was 6.334
finish time 2017-05-12 20:00:00
Profit on this trade: 106.666
starting time 2017-05-14 05:00:00
Buying BTC @ 1726.0
Selling BTC @ 1679.0
Fee for this was 6.81
finish time 2017-05-15 09:00:00
Profit on this trade: -53.81
starting time 2017-05-17 02:00:00
Buying BTC @ 1722.94
Selling BTC @ 2407.6
Fee for this was 8.26108
finish time 2017-05-26 07:00:00
Profit on this trade: 676.39892
starting time 2017-05-28 15:00:00
Buying BTC @ 2010.0
Selling BTC @ 2156.03
Fee for this was 8.33206
finish time 2017-05-31 05:00:00
Profit on this trade: 137.69794
starting time 2017-06-01 02:00:00
Buying BTC @ 2258.4
Selling BTC @ 2622.0
Fee for this was 9.7608
finish time 2017-06-07 21:00:00
Profit on this trade: 353.8392
starting time 2017-06-09 09:00:00
Buying BTC @ 2713.99
Selling BTC @ 2720.0
Fee for this was 10.86798
finish time 2017-06-12 14:00:00
Profit on this trade: -4.85798
starting time 2017-06-14 

In [11]:
# In this cell I calculate the entry and exit signals of the strategy. When the fast moving average, cross up the slow moving average, an entry signal is triger. 
# The entry_signals variable take the value of 1 if this set up happened ans zero otherwise.
bitcoin_data["entry_signals"] = np.where((bitcoin_data["20d"] > bitcoin_data["50d"]) & (bitcoin_data["20d"].shift(1) < bitcoin_data["50d"].shift(1)),1,0)
bitcoin_data["exit_signals"] = np.where((bitcoin_data["20d"] < bitcoin_data["50d"]),1,0)

In [12]:
bitcoin_data

,Date,open,high,low,close,volumefrom,volumeto,20d,50d,H-L,H-YC,L-YC,filtered_exit,entry_signals,exit_signals
0,2017-05-06 01:00:00,1443.00,1454.07,1437.11,1453.20,144.44,208839.37,NaN,NaN,16.96,NaN,NaN,NaN,0,0
1,2017-05-06 02:00:00,1453.20,1457.00,1445.40,1446.32,180.41,262186.53,NaN,NaN,11.60,3.80,7.80,NaN,0,0
2,2017-05-06 03:00:00,1446.32,1455.00,1445.50,1450.10,63.11,91504.70,NaN,NaN,9.50,8.68,0.82,NaN,0,0
3,2017-05-06 04:00:00,1450.10,1460.22,1450.01,1460.15,290.63,423495.61,NaN,NaN,10.21,10.12,0.09,NaN,0,0
4,2017-05-06 05:00:00,1460.15,1460.22,1449.00,1450.00,120.53,175248.11,NaN,NaN,11.22,0.07,11.15,NaN,0,0
5,2017-05-06 06:00:00,1450.00,1456.77,1447.12,1450.00,142.96,207422.34,NaN,NaN,9.65,6.77,2.88,NaN,0,0
6,2017-05-06 07:00:00,1450.00,1460.00,1450.00,1458.55,228.51,332400.35,NaN,NaN,10.00,10.00,0.00,NaN,0,0
7,2017-05-06 08:00:00,1458.55,1465.00,1454.80,1461.00,159.15,232196.16,NaN,NaN,10.20,6.45,3.75,NaN,0,0
8,2017-05-06 09:00:00,1461.00,1468.15,1456.70,1458.50,130.25,190330.49,NaN,NaN,11.45,7.15,4.30,NaN,0,0
9,2017-05-06 10:00:00,1458.50,1458.50,1445.00,1452.30,169.96,246763.57,NaN,NaN,13.50,0.00,13.50,NaN,0,0


In [13]:
# This lines yield that there are a lof of exit_signals. We have to filtered the exit_signals variable in order to utilize this
# variable if and only if there is a trade in place. 
print(len(bitcoin_data[bitcoin_data["entry_signals"] ==1]))
print(len(bitcoin_data[bitcoin_data["exit_signals"]==1]))

16
758


In [14]:
# Create a series pandas filtered_exit with nan values, to populate afterwards with the logic of the order management.
bitcoin_data["filtered_exit"] = np.nan

In [15]:
for i, data in bitcoin_data.iterrows():
    holding_status = (bitcoin_data.loc[:(i-1), 'entry_signals'] - bitcoin_data.loc[:(i-1), 'filtered_exit']).sum()
    
    if bitcoin_data["exit_signals"][i] == 0 or i == 0:
        bitcoin_data.loc[i, 'filtered_exit'] = 0
    elif bitcoin_data["exit_signals"][i] == 1:
        if holding_status == 0:
            bitcoin_data.loc[i, 'filtered_exit'] = 0
        elif holding_status == 1:
            bitcoin_data.loc[i, 'filtered_exit'] = 1

In [16]:
# With the for loop before, we get an exit signal for each of the entry signals, and therefore we get all the accurate signals within the strategy
print(len(bitcoin_data[bitcoin_data["entry_signals"] ==1]))
print(len(bitcoin_data[bitcoin_data["filtered_exit"]==1]))

16
16


In [21]:
# Create the trades data frame with some information of the trades in the strategy 

trades = pd.concat([pd.DataFrame({"Date": bitcoin_data.loc[bitcoin_data["entry_signals"]==1,"Date"], "Price": bitcoin_data.loc[bitcoin_data["entry_signals"]==1,"close"],
                                   "Regime":bitcoin_data.loc[bitcoin_data["entry_signals"]==1,"entry_signals"],
                                   "Signal": "Buy"}),
                    pd.DataFrame({"Date": bitcoin_data.loc[bitcoin_data["filtered_exit"]==1,"Date"], "Price":bitcoin_data.loc[bitcoin_data["filtered_exit"]==1,"close"],
                                 "Regime":bitcoin_data.loc[bitcoin_data["filtered_exit"]==1,"filtered_exit"],
                                 "Signal":"Sell"}),])
trades.sort_index(inplace=True)
trades

,Date,Price,Regime,Signal
196,2017-05-14 05:00:00,1726.00,1.0,Buy
224,2017-05-15 09:00:00,1679.00,1.0,Sell
265,2017-05-17 02:00:00,1722.94,1.0,Buy
486,2017-05-26 07:00:00,2407.60,1.0,Sell
542,2017-05-28 15:00:00,2010.00,1.0,Buy
604,2017-05-31 05:00:00,2156.03,1.0,Sell
625,2017-06-01 02:00:00,2258.40,1.0,Buy
788,2017-06-07 21:00:00,2622.00,1.0,Sell
824,2017-06-09 09:00:00,2713.99,1.0,Buy
901,2017-06-12 14:00:00,2720.00,1.0,Sell


In [22]:
# Make and index timestamp with the Date Column of the trades dataframe
trades.index = pd.to_datetime(trades["Date"],unit="s")
trades = trades.drop("Date",axis=1)


In [23]:
# Create the trades_long_profits that show the datetime at the entry moment(index) of the trade, the end time of the trade, the price at which
# the trades was trigered and the profit of the trade
trades_long_profits = pd.DataFrame({
        
    "End_date":  trades["Price"].loc[trades.loc[trades["Signal"].shift(1)=="Buy"].index].index,
    "Price":trades.loc[(trades["Signal"]=="Buy") & trades["Regime"]==1,"Price"],
    "Profit":pd.Series(trades["Price"] - trades["Price"].shift(1) - (0.002*(trades["Price"] + trades["Price"].shift(1)))).loc[trades.loc[(trades["Signal"].shift(1)=="Buy")
        &(trades["Regime"].shift(1)==1)].index].tolist(),
                       })
trades_long_profits

,End_date,Price,Profit
Date,,,
2017-05-14 05:00:00,2017-05-15 09:00:00,1726.00,-53.81000
2017-05-17 02:00:00,2017-05-26 07:00:00,1722.94,676.39892
2017-05-28 15:00:00,2017-05-31 05:00:00,2010.00,137.69794
2017-06-01 02:00:00,2017-06-07 21:00:00,2258.40,353.83920
2017-06-09 09:00:00,2017-06-12 14:00:00,2713.99,-4.85798
2017-06-14 05:00:00,2017-06-14 19:00:00,2711.00,-162.13082
2017-06-16 16:00:00,2017-06-18 19:00:00,2488.84,-57.69968
2017-06-19 20:00:00,2017-06-22 08:00:00,2498.05,130.67590
2017-06-23 00:00:00,2017-06-24 09:00:00,2689.96,-30.97932


In [24]:
investment = 100000
trades_long_profits["portfolio_value"] =  investment  + trades_long_profits["Profit"].cumsum()
trades_long_profits["returns"] = ((trades_long_profits["portfolio_value"] / trades_long_profits["portfolio_value"].shift(1)) -1) * 100


In [25]:
import math
def create_drawdown(pnl):
    hwm =[0]
    idx = pnl.index
    drawdown = pd.Series(index = idx)
    duration = pd.Series(index = idx)
    for t in range(1, len(idx)):
        hwm.append(max(hwm[t-1], pnl[t]))
        drawdown[t] = (hwm[t] -pnl[t]) 
        duration[t] = (0 if drawdown[t] == 0 else duration[t-1] +1)
    return drawdown, drawdown.max() , duration.max() 

trades_long_profits["drawdown"] = create_drawdown(trades_long_profits["returns"])[0]
trades_long_profits["net_profit"] = trades_long_profits["portfolio_value"] - 100000
trades_long_profits

,End_date,Price,Profit,portfolio_value,returns,drawdown,net_profit
Date,,,,,,,
2017-05-14 05:00:00,2017-05-15 09:00:00,1726.00,-53.81000,99946.19000,NaN,NaN,-53.81000
2017-05-17 02:00:00,2017-05-26 07:00:00,1722.94,676.39892,100622.58892,0.676763,0.000000,622.58892
2017-05-28 15:00:00,2017-05-31 05:00:00,2010.00,137.69794,100760.28686,0.136846,0.539917,760.28686
2017-06-01 02:00:00,2017-06-07 21:00:00,2258.40,353.83920,101114.12606,0.351169,0.325594,1114.12606
2017-06-09 09:00:00,2017-06-12 14:00:00,2713.99,-4.85798,101109.26808,-0.004804,0.681568,1109.26808
2017-06-14 05:00:00,2017-06-14 19:00:00,2711.00,-162.13082,100947.13726,-0.160352,0.837115,947.13726
2017-06-16 16:00:00,2017-06-18 19:00:00,2488.84,-57.69968,100889.43758,-0.057158,0.733921,889.43758
2017-06-19 20:00:00,2017-06-22 08:00:00,2498.05,130.67590,101020.11348,0.129524,0.547239,1020.11348
2017-06-23 00:00:00,2017-06-24 09:00:00,2689.96,-30.97932,100989.13416,-0.030666,0.707430,989.13416


In [26]:
Total_Gross_Win = trades_long_profits[trades_long_profits["Profit"] > 0].sum()["Profit"]
Total_Gross_Loss = trades_long_profits[trades_long_profits["Profit"] < 0].sum()["Profit"]
Profit_Factor = abs(Total_Gross_Win / Total_Gross_Loss)
Total_Net_Profit = (Total_Gross_Win + Total_Gross_Loss)
Number_Winning_Trades = trades_long_profits[trades_long_profits["Profit"] > 0].count()["Profit"]
Number_Lossing_Trades = trades_long_profits[trades_long_profits["Profit"] < 0].count()["Profit"]
Percentage_Profitable_Trades = (Number_Winning_Trades) / (Number_Winning_Trades + Number_Lossing_Trades)
Largest_Profitable_Trade = max(trades_long_profits["Profit"])
Largest_Losing_Trade = min(trades_long_profits["Profit"])
Max_Draw_Down = trades_long_profits["drawdown"].max()

In [27]:
def sharpe_ratio(returns):
    return (returns.mean() / returns.std()) * np.sqrt(len(trades_long_profits))
Sharpe_ratio = sharpe_ratio(trades_long_profits["returns"])

In [28]:
print("Total_Gross_Win:", Total_Gross_Win )
print("Total_Gross_Loss:", Total_Gross_Loss)
print("Profit_Factor:", Profit_Factor)
print("Total_Net_Profit", Total_Net_Profit)
print("Number_Winning_Trades:", Number_Winning_Trades)
print("Average_Winning_Trade:",Total_Gross_Win / Number_Winning_Trades)
print("Number_Lossing_Trades:", Number_Lossing_Trades)
print("Average_Lossing_Trade:", Total_Gross_Loss / Number_Lossing_Trades)
print("Percentage_Profitable_Trades:", Percentage_Profitable_Trades)
print("Largest_Profitable_Trade:", Largest_Profitable_Trade)
print("Largest_Losing_Trade:", Largest_Losing_Trade)
print("Max_Draw_Down:", Max_Draw_Down)
print("Sharpe_ratio:",Sharpe_ratio)

Total_Gross_Win: 1905.39072
Total_Gross_Loss: -670.5537
Profit_Factor: 2.84151846452
Total_Net_Profit 1234.83702
Number_Winning_Trades: 6
Average_Winning_Trade: 317.56512
Number_Lossing_Trades: 10
Average_Lossing_Trade: -67.05537
Percentage_Profitable_Trades: 0.375
Largest_Profitable_Trade: 676.39892
Largest_Losing_Trade: -162.13082
Max_Draw_Down: 0.837115171717
Sharpe_ratio: 1.36511736082


The strategy shows good result with a Total Net Profit of U$S 1234. With this strategy we will win 6 times out of 16 times, but the average wining trade(U$S317) is much higher than the average lossing trades(U$S-67)

The sharpe ratio is satisfactory and also it is possible to optimize the strategy by apply some risk management techniques that increment or diminish the size of the trade in behalf the market condition. 
The only matter of this strategy is that at some point we have to bear with an important drawdawn, there are some lossing trades of U$S-162, and U$S-138.